<a href="https://colab.research.google.com/github/jinensetpal/ArchiMeDe/blob/master/augment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!curl https://rclone.org/install.sh | sudo bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4491  100  4491    0     0   4844      0 --:--:-- --:--:-- --:--:--  4844
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    15  100    15    0     0     16      0 --:--:-- --:--:-- --:--:--    16

The latest version of rclone rclone v1.55.1 is already installed.



In [ ]:
# !rclone config ; cp /root/.config/rclone/rclone.conf .
!wget https://8d121c98e8e8.ngrok.io/rclone.conf ; mkdir -p /root/.config/rclone/ ; cp rclone.conf /root/.config/rclone/

--2021-06-25 03:15:50--  https://8d121c98e8e8.ngrok.io/rclone.conf
Resolving 8d121c98e8e8.ngrok.io (8d121c98e8e8.ngrok.io)... 3.22.30.40, 2600:1f16:d83:1202::6e:2
Connecting to 8d121c98e8e8.ngrok.io (8d121c98e8e8.ngrok.io)|3.22.30.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3686 (3.6K) [text/plain]
Saving to: ‘rclone.conf’

rclone.conf         100%[===================>]   3.60K  --.-KB/s    in 0s      

2021-06-25 03:15:51 (56.3 MB/s) - ‘rclone.conf’ saved [3686/3686]



In [6]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount colab: /content/onedrive &

mkdir: cannot create directory ‘/content/onedrive’: File exists
nohup: appending output to 'nohup.out'


In [ ]:
!curl 'https://www.cityscapes-dataset.com/file-handling/?packageID=3' # pruned request

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7G  100 10.7G    0     0  24.9M      0  0:07:23  0:07:23 --:--:-- 24.3M


In [ ]:
!cp cityscapes.zip /content/onedrive/projects/deeplab-panoptic/

In [ ]:
!rm -r /content/onedrive/projects/deeplab-panoptic/leftImg8bit/

In [ ]:
!unzip -n /content/onedrive/projects/deeplab-panoptic/cityscapes.zip -d /content/onedrive/projects/deeplab-panoptic/

Archive:  /content/onedrive/projects/deeplab-panoptic/cityscapes.zip
 extracting: /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/zurich/zurich_000065_000019_leftImg8bit.png  
 extracting: /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/zurich/zurich_000003_000019_leftImg8bit.png  
 extracting: /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/zurich/zurich_000058_000019_leftImg8bit.png  
 extracting: /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/zurich/zurich_000018_000019_leftImg8bit.png  
 extracting: /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/zurich/zurich_000057_000019_leftImg8bit.png  
 extracting: /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/zurich/zurich_000086_000019_leftImg8bit.png  
 extracting: /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/zurich/zurich_000063_000019_leftImg8bit.png  
 extracting: /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/zurich/zu

In [ ]:
!mkdir -p /content/onedrive/projects/deeplab-panoptic/data/mask
!cp /content/onedrive/projects/deeplab-panoptic/gtFine/train/**/*_color.png /content/onedrive/projects/deeplab-panoptic/data/mask

In [ ]:
!mkdir -p /content/onedrive/projects/deeplab-panoptic/data/img
!cp /content/onedrive/projects/deeplab-panoptic/leftImg8bit/train/**/*.png /content/onedrive/projects/deeplab-panoptic/data/img

In [ ]:
import os
import random
import pickle
from numpy import linspace
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# we create two instances with the same arguments
data_gen_args = dict(rescale=random.choice(linspace(.5, 2.0, 16)),
                     horizontal_flip=True)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 836

image_generator = image_datagen.flow_from_directory(
    '/content/onedrive/projects/deeplab-panoptic/data/img',
    class_mode=None,
    seed=seed)
mask_generator = mask_datagen.flow_from_directory(
    '/content/onedrive/projects/deeplab-panoptic/data/mask',
    class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

pickle.dump(train_generator,
            open(os.path.join('/content/onedrive/projects/deeplab-panoptic/train_generator.pkl'), 'wb'),
            protocol=4)

KeyboardInterrupt: ignored

In [ ]:
import cv2
import random
from numpy import expand_dims, linspace
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img, ImageDataGenerator

def augment(filepath, rescale, spec, seed):
  img = load_img(filepath)
  data = img_to_array(img)
  samples = expand_dims(data, 0)
  gen = ImageDataGenerator(ImageDataGenerator,
                           rescale=rescale,
                           horizontal_flip=True)
  it = gen.flow(samples, batch_size=1, seed=seed)
  batch = it.next()
  image = batch[0].astype('float32')
  image = cv2.resize(image, (2049, 1025))
  save_img('/content/onedrive/projects/deeplab-panoptic/augmented/' + spec + '/' + filepath.split('/')[-1][:-4] + '_' + str(i) + '.png', image)

In [8]:
import os 
import glob 
import random
from tqdm import tqdm

base = '/content/onedrive/projects/deeplab-panoptic/'
for num, (h, x, y) in tqdm(enumerate(zip([os.listdir(base + 'leftImg8bit/train'), os.listdir(base + 'gtFine/train')], ['leftImg8bit/train/', 'gtFine/train/'], ['/*.png', "/*color.png"])), position=0):
  random.seed(836)
  for i in tqdm(h, position=0, leave=False):
    for j in tqdm(glob.glob(base + x + i + y), position=0, leave=False):
      if len(glob.glob(base + 'augmented/' + x.split('/')[0] + '/' + j.split('/')[-1][:-4] + '_*')) == 0:
        rescale = random.choice(linspace(0.5, 2, 16))
        augment(j, rescale, x.split('/')[0], num)
        if x.split('/')[0] == 'gtFine':
          augment(j[:-9] + 'instanceIds.png', rescale, x.split('/')[0], num)
          augment(j[:-9] + 'labelIds.png', rescale, x.split('/')[0], num)
        print('Augmented: Image', j)

  0%|          | 0/119 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
  1%|          | 1/119 [00:02<04:38,  2.36s/it]

Augmented: Image /content/onedrive/projects/deeplab-panoptic/gtFine/train/jena/jena_000000_000019_gtFine_color.png


  2%|▏         | 2/119 [00:03<04:03,  2.08s/it]

Augmented: Image /content/onedrive/projects/deeplab-panoptic/gtFine/train/jena/jena_000001_000019_gtFine_color.png


  3%|▎         | 3/119 [00:05<03:40,  1.90s/it]

...

Augmented: Image /content/onedrive/projects/deeplab-panoptic/gtFine/train/zurich/zurich_000119_000019_gtFine_color.png


 99%|█████████▉| 121/122 [04:07<00:01,  1.67s/it]

Augmented: Image /content/onedrive/projects/deeplab-panoptic/gtFine/train/zurich/zurich_000120_000019_gtFine_color.png


2it [50:31, 1515.96s/it]

Augmented: Image /content/onedrive/projects/deeplab-panoptic/gtFine/train/zurich/zurich_000121_000019_gtFine_color.png
